In [1]:
#!pip install wrds

In [2]:
import wrds
import pandas as pd
#import sqlalchemy
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

In [3]:
#db = wrds.Connection(wrds_username='mweigel00')
#db.create_pgpass_file()

In [4]:
#db.close() #verify that my pass works without entering the password all the time
db = wrds.Connection(wrds_username='mweigel00') #it works!

Loading library list...
Done


In [25]:
#first pull the part of the data that is from 2021
sql_query_1 = """
SELECT
  *
FROM
  (
    SELECT
      *,
      RANK() OVER (PARTITION BY sym_root, date, EXTRACT (HOUR FROM time_m), EXTRACT (MINUTE FROM time_m) ORDER BY time_m, time_m_nano) AS time_rank
    FROM
      taqm_2021.complete_nbbo_2021
    WHERE
      sym_root in ('F') AND
      date BETWEEN '20210930' AND '20220930'
  ) a
WHERE
  time_rank = 1
ORDER BY
  time_m, 
  sym_root;
"""

# Assuming you have a connection object, like conn
result_2021 = db.raw_sql(sql_query_1, date_cols=['date'])


# now just pull the part of the data that is from 2022

sql_query_2 = """
SELECT
  *
FROM
  (
    SELECT
      *,
      RANK() OVER (PARTITION BY sym_root, date, EXTRACT (HOUR FROM time_m), EXTRACT (MINUTE FROM time_m) ORDER BY time_m, time_m_nano) AS time_rank
    FROM
      taqm_2022.complete_nbbo_2022
    WHERE
      sym_root in ('F') AND
      date BETWEEN '20210930' AND '20220930'
  ) a
WHERE
  time_rank = 1
ORDER BY
  time_m, 
  sym_root;
"""

result_2022 = db.raw_sql(sql_query_2, date_cols=['date'])


In [26]:
# Concatenate the DataFrames
concatenated_df = pd.concat([result_2021, result_2022], ignore_index=True)
concatenated_df["date"].value_counts()

2022-02-04    951
2022-01-05    942
2022-01-20    941
2021-10-28    929
2022-01-06    925
             ... 
2022-06-08    674
2022-08-24    671
2022-06-01    669
2021-09-30    660
2022-09-07    652
Name: date, Length: 253, dtype: int64

In [27]:
sorted_data = concatenated_df.sort_values(by=['date', 'time_m'], ascending=[True, True])
sorted_data = sorted_data.dropna(subset = ["best_ask", "best_bid"]).reset_index().drop(columns= ["index"]) # sometimes there is either no ask or bid, in this case we cannot assume to be able to make a trade
sorted_data

,date,time_m,time_m_nano,sym_root,sym_suffix,qu_cond,natbbo_ind,qu_source,nbbo_qu_cond,best_bid,best_bidsizeshares,best_ask,best_asksizeshares,time_rank
0,2021-09-30,04:00:00.016496,128,F,None,R,U,C,None,11.54,200.0,14.48,2600.0,1
1,2021-09-30,04:02:06.578113,24,F,None,R,U,C,None,14.36,400.0,14.40,200.0,1
2,2021-09-30,04:03:03.544752,128,F,None,R,U,C,None,14.42,600.0,14.45,700.0,1
3,2021-09-30,04:04:51.459206,632,F,None,R,U,C,None,14.42,600.0,14.45,600.0,1
4,2021-09-30,04:05:02.407123,200,F,None,R,U,C,None,14.42,400.0,14.45,600.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200675,2022-09-30,19:53:00.422458,112,F,None,R,U,C,None,11.20,5300.0,11.21,16500.0,1
200676,2022-09-30,19:54:00.214454,272,F,None,R,U,C,None,11.20,8500.0,11.21,16500.0,1
200677,2022-09-30,19:58:47.250492,416,F,None,R,U,C,None,11.20,800.0,11.21,16000.0,1
200678,2022-09-30,19:59:22.183307,264,F,None,R,U,C,None,11.20,800.0,11.21,15800.0,1


In [28]:
sorted_data["sym_root"][1]

'F'

In [29]:
sorted_data.to_csv('Minute_data_'+str(sorted_data["sym_root"][1])+'.csv', index=False) # save data